In [0]:
import os
import requests
from bs4 import BeautifulSoup
import csv
import re

import pandas as pd
import numpy as np


from google.colab import files


# 2k ratings data


In [0]:
# base URL of the site
url="https://nba2k19.2kratings.com"


# robots.txt

def check_robots_txt(url):
  url_robots = url + '/robots.txt'
  page = requests.get(url_robots)
  soup = BeautifulSoup(page.content, 'lxml')
  print(soup.prettify())
  
  
check_robots_txt(url) 

<html>
 <body>
  <p>
   User-Agent: *
Disallow: /wp-admin/
Sitemap: https://nba2k19.2kratings.com/page-sitemap.xml
Sitemap: https://nba2k19.2kratings.com/team-sitemap.xml
Sitemap: https://nba2k19.2kratings.com/player-sitemap1.xml
Sitemap: https://nba2k19.2kratings.com/player-sitemap2.xml
Sitemap: https://nba2k19.2kratings.com/player-sitemap3.xml
Crawl-delay: 10
  </p>
 </body>
</html>


In [0]:
# modify the function to add the results to an existing df

def get_data(url):
  
  # request to the page
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'lxml')
  
  
  
  # get the data for each player in the table
  
  # table containing all players
  table = soup.find("tbody")





  for player in table.find_all("tr"):

    # name of the player
    name = player.find_all("td")[1].a.text
    names.append(name)

    # name of the team
    teams.append(dict_team_url[url])
    
#     team = player.find_all("td")[1].small.text
#     teams.append(team)

    # pos
    pos = player.find_all("td")[3].text
    positions.append(pos)

    # height
    height =  player.find_all("td")[4].text
    heights.append(height)

    # rating
    rating = player.find_all("td")[2].span.text
    ratings.append(rating)

    # brand
    brand = player.find_all("td")[5].text
    brands.append(brand)


    # check if all star 
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-star"})) > 0:
      all_star = 1
    else: 
      all_star = 0
    all_stars.append(all_star)

    # check if injured
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-plus-circle"})) > 0:
      injured = 1
    else: 
      injured = 0
    injury.append(injured)

    # check if two-way
    if len(player.find_all("td")[1].find_all("i", {"class": "fa fa-arrows-h"})) > 0:
      two_way = 1
    else: 
      two_way = 0
    two_ways.append(two_way)


    # check if rookie
    if len(player.find_all("td")[1].find_all("small")) == 1:
      rookie = 1
    else:
      rookie = 0
    rookies.append(rookie)    


In [0]:
#start getting  list of the url where we can obtain all the data: one URL per team
url = 'https://nba2k19.2kratings.com/current-teams-on-nba-2k19'



# request to the page
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

teams = soup.find("ul", {"class": "list-group"})

url_teams = []
team_list = []

for team in teams.find_all('li'):
  url_teams.append(team.a['href'])
  team_list.append(team.a.text)
  


dict_team_url = {url_teams[i]:team_list[i] for i in range(0, len(team_list))}

print(dict_team_url)

{'https://nba2k19.2kratings.com/team/atlanta-hawks': 'Atlanta Hawks', 'https://nba2k19.2kratings.com/team/boston-celtics': 'Boston Celtics', 'https://nba2k19.2kratings.com/team/brooklyn-nets': 'Brooklyn Nets', 'https://nba2k19.2kratings.com/team/charlotte-hornets': 'Charlotte Hornets', 'https://nba2k19.2kratings.com/team/chicago-bulls': 'Chicago Bulls', 'https://nba2k19.2kratings.com/team/cleveland-cavaliers': 'Cleveland Cavaliers', 'https://nba2k19.2kratings.com/team/dallas-mavericks': 'Dallas Mavericks', 'https://nba2k19.2kratings.com/team/denver-nuggets': 'Denver Nuggets', 'https://nba2k19.2kratings.com/team/detroit-pistons': 'Detroit Pistons', 'https://nba2k19.2kratings.com/team/golden-state-warriors': 'Golden State Warriors', 'https://nba2k19.2kratings.com/team/houston-rockets': 'Houston Rockets', 'https://nba2k19.2kratings.com/team/indiana-pacers': 'Indiana Pacers', 'https://nba2k19.2kratings.com/team/los-angeles-clippers': 'Los Angeles Clippers', 'https://nba2k19.2kratings.com/t

In [0]:
names = []
teams = []
positions = []
heights = []
ratings = []
brands = []
injury = []
two_ways = []
all_stars = []
rookies = []

for url in url_teams:
  get_data(url)
  
# create dataframe
ratings_19 = pd.DataFrame({'name': names, 'team': teams, 'position': positions, 
              'height' : heights, 'brand' : brands,
              'all_star': all_stars, 'injured' : injury, 'two_way' : two_ways, 'rookie': rookies,  'rating': ratings})  

In [0]:
# Creamos el fichero
ratings_19.to_csv("2k19_ratings.csv", index=False)

# descargamos en formato .csv
files.download("2k19_ratings.csv")

# Stats data

In [0]:
# base URL of the site
url="https://www.basketball-reference.com"


# robots.txt
check_robots_txt(url) 

In [0]:
def get_stats(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'lxml')


  col_names = []
  for i in range(1,len(soup.tbody.find_all('tr')[1].find_all('td'))+1):
        col_names.append(soup.thead.find_all('th')[i].text.replace('%','.pct'))

  players = soup.tbody.find_all('tr')
  stats_df = pd.DataFrame(np.zeros((0,len(soup.tbody.find_all('tr')[1].find_all('td')))))

  # loop through the players
  for player in players:

    if player['class'] != ['thead']:

      # create list for every player
      player_as_list = []

      #loop through the player's stats
      for col in range(0,len(soup.tbody.find_all('tr')[1].find_all('td'))):
        player_as_list.append(player.find_all('td')[col].text)

      # obtain dataframe  
      df = pd.DataFrame(player_as_list)
      # append to the whole dataframe; transposed
      stats_df = stats_df.append(df.T)

  # modify column names and index
  stats_df.columns=col_names
  stats_df.index = np.arange(len(stats_df))
  
  # change blank spaces for NAs:
  stats_df = stats_df.replace(r'^\s*$', np.nan, regex=True)
  
  return stats_df

## Per Game stats

In [0]:
stats_1718 = get_stats('https://www.basketball-reference.com/leagues/NBA_2018_per_game.html')

## Advanced stats

In [0]:
stats_1718_adv = get_stats('https://www.basketball-reference.com/leagues/NBA_2018_advanced.html')

## Full stats: per game plus advanced

In [0]:
stats_1718_full = pd.merge(stats_1718, stats_1718_adv, how='inner', on=["Player", 'Pos', 'Age', 'Tm', 'G'])

stats_1718_full.columns = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MPG', 'FG', 'FGA', 'FG.pct',
       '3P', '3PA', '3P.pct', '2P', '2PA', '2P.pct', 'eFG.pct', 'FT', 'FTA',
       'FT.pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'MP_Total', 'PER', 'TS.pct', '3PAr', 'FTr', 'ORB.pct', 'DRB.pct', 'TRB.pct',
       'AST.pct', 'STL.pct', 'BLK.pct', 'TOV.pct', 'USG.pct', 'empty1', 'OWS',
       'DWS', 'WS', 'WS/48', 'empty2', 'OBPM', 'DBPM', 'BPM', 'VORP']
# drop columns with NA created with the scraping part
stats_1718_full  = stats_1718_full.drop(['empty1', 'empty2'], axis = 1)

In [0]:
# Creamos el fichero
stats_1718_full.to_csv("full_stats_1718.csv", index=False)

# descargamos en formato .csv
files.download("full_stats_1718.csv")